In [1]:
import syft as sy
import numpy as np
from syft.core.node.worker import Worker
from syft.core.node.new.user_code import SubmitUserCode
from syft.core.node.new.policy import SubmitUserPolicy

/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo


In [2]:
worker = Worker()

> Starting Worker: Priceless Altman - 21ca746bca994631a8efc1dc13435957 - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis")

Logged into Priceless Altman as <info@openmined.org>


In [4]:
from pydantic import BaseModel
from typing import List, Dict, Any

class A(BaseModel):
    class Config:
        arbitrary_types_allowed = True

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(**kwargs)
        print(self.__post_init__)
        print(args)
        print(kwargs)
        self.__post_init__()
        
    def __post_init__(self):
        pass
    
    def update() -> None:
        raise NotImplementedError
    
class B(A):
    init_args: Dict[str, Any]={}
    kwargs: Dict[str, Any]={}

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(**kwargs)
        self.init_args = args
        self.kwargs = kwargs

    
class C(B):
    n_calls: int
    downloadable_output_args: List[str]
    
    
    
# B()     
c = C(n_calls=1, downloadable_output_args=[])
c.kwargs

<bound method A.__post_init__ of C(init_args={}, kwargs={}, n_calls=1, downloadable_output_args=[])>
()
{'n_calls': 1, 'downloadable_output_args': []}


{'n_calls': 1, 'downloadable_output_args': []}

In [5]:
c.__post_init__

<bound method A.__post_init__ of C(init_args=(), kwargs={'n_calls': 1, 'downloadable_output_args': []}, n_calls=1, downloadable_output_args=[])>

In [6]:
# from syft.core.common.serde.serializable import serializable
from syft.core.node.new.user_code import ExactMatch, SingleExecutionExactOutput, OutputPolicy, CustomOutputPolicy
from syft.core.node.new.action_object import ActionObject

class RepeatedCallPolicy(CustomOutputPolicy):
    # from syft.core.node.new.action_object import ActionObject
    __canonical_name__ = "RepeatedCallPolicy"
    from syft.core.node.new.syft_object import SYFT_OBJECT_VERSION_1

    __version__ = SYFT_OBJECT_VERSION_1
    from typing import Dict
    from typing import Any
    n_calls: int
    downloadable_output_args: List[str]
    state: Dict[Any, Any] = {}
    # _init_args: Dict[str, Any]
    # _kwargs: Dict[str, Any]
    
    __attr_allowlist__ = [
        "n_calls",
        "downloadable_output_args",
        "_init_args",
        "_kwargs"
    ]
    
    # __fields_set__ = [
    #     "n_calls",
    #     "downloadable_output_args",
    # ]
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        super().__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}

    # def __repr__(self) -> str:
    #     return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    # def update() -> None:

    # def __post_init__(self, *args, **kwargs) -> None:
    #     self._init_args = args
    #     self._kwargs = kwargs

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
    
    

In [7]:
policy = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y'])
print(policy.init_args, policy.kwargs)
obj = ActionObject.from_obj({'y': [1,2,3]})
policy.apply_output(obj)

() {'n_calls': 1, 'downloadable_output_args': ['y']}


{'y': [1, 2, 3]}

In [8]:
import numpy as np
from syft.core.node.new.twin_object import to_action_object

x = np.array([1,2,3])
x_pointer = to_action_object(x)
client.api.services.action.save(x_pointer)

<class 'syft.core.node.new.response.SyftSuccess'>: <class 'syft.core.node.new.numpy.NumpyArrayObject'> saved

In [14]:
from inspect import getmro
obj = RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']).__class__
getmro(obj)

(__main__.RepeatedCallPolicy,
 syft.core.node.new.user_code.CustomOutputPolicy,
 syft.core.node.new.user_code.OutputPolicy,
 syft.core.node.new.syft_object.SyftObject,
 syft.core.node.new.syft_object.SyftBaseObject,
 pydantic.main.BaseModel,
 pydantic.utils.Representation,
 syft.core.node.new.syft_object.SyftObjectRegistry,
 object)

In [17]:
obj.__object_version_registry__['RepeatedCallPolicy_1']

TypeError: eval() arg 1 must be a string, bytes or code object

In [10]:
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=RepeatedCallPolicy(n_calls=1, downloadable_output_args=['y']),
)
def func(x):
    return {"y": x}


SubmitUserPolicy
('x',)


In [11]:
client.api.services.code.request_code_execution(func)

Noerr before code to usercode
['x']
syft.core.node.new.policy.SubmitUserPolicy
NodeType.DOMAIN
dict_keys([NodeView(node_name='Priceless Altman', verify_key=b98587bb6c1fa231f05839d91f8b533d904838888becbdd01c22c96377e8e99a)])
node_name='Priceless Altman' verify_key=b98587bb6c1fa231f05839d91f8b533d904838888becbdd01c22c96377e8e99a
Noerr before policy transformations
Noerr


```python
class Request:
  id: str = 2c9e626577c145a3abb353e24988bfa4
  requesting_user_verify_key: str = b98587bb6c1fa231f05839d91f8b533d904838888becbdd01c22c96377e8e99a
  approving_user_verify_key: str = None
  request_time: str = 2023-03-19 16:11:11
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 21ca746bca994631a8efc1dc13435957
  request_hash: str = "1928cdfe892eac0a4482c30ec11c642d518ab7dbc90da84832b369ffcb370ec9"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange]

```

In [12]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Priceless Altman as <info@openmined.org>


In [13]:
request = client.notifications[0].link
func = request.changes[0].link
result = func.unsafe_function(x=x_pointer)
# func.output_policy
final_result = request.accept_by_depositing_result(result) 
final_result

['Config', '__abstractmethods__', '__annotations__', '__attr_repr_cols__', '__attr_searchable__', '__attr_state__', '__attr_unique__', '__canonical_name__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__object_transform_registry__', '__object_version_registry__', '__post_init__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__serde_overrides__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots_

execute_byte_code failed Duplicate mapping for RepeatedCallPolicy_1 and <class 'syft.core.node.new.policy.RepeatedCallPolicy'>
execute_byte_code failed name 'RepeatedCallPolicy' is not defined


TypeError: None argument after ** must be a mapping, not NoneType

In [ ]:
func.output_policy

```python
class SingleExecutionExactOutput:
  id: str = 4329e8bff38246e3b5b5702066d75146
  outputs: str = []
  state_type: str = <class 'syft.core.node.new.user_code.OutputPolicyStateExecuteOnce'>
  init_args: str = ()
  kwargs: str = {'id': <UID: 4329e8bff38246e3b5b5702066d75146>, 'init_args': (), 'kwargs': {'id': <UID: 4329e8bff38246e3b5b5702066d75146>, 'init_args': (), 'kwargs': {}, 'outputs': [], 'state_type': <class 'syft.core.node.new.user_code.OutputPolicyStateExecuteOnce'>}, 'outputs': [], 'state_type': <class 'syft.core.node.new.user_code.OutputPolicyStateExecuteOnce'>}

```

In [ ]:
res = client.api.services.code.func(x=x_pointer)
print(res)
print(type(res))

{'x': array([1, 2, 3])}
message='Policy is no longer valid. count: 1 >= limit: 1'
<class 'syft.core.node.new.response.SyftError'>


In [ ]:
s

NameError: name 's' is not defined

In [ ]:
client.api.services.policy.get_all()

,type,id
0,syft.core.node.new.policy.UserPolicy,0c381e061c4c44968618abcf22b8ec9e


In [ ]:
output_policy = client.api.services.policy.get_all()[0]


In [ ]:
client.api.services.policy.get_all()

,type,id
0,syft.core.node.new.policy.UserPolicy,0c381e061c4c44968618abcf22b8ec9e


In [ ]:
@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}


UserPolicy
('x',)


In [ ]:
client.api.services.code.request_code_execution(func_plus_one)

Noerr before code to usercode
['x']
c6c4a8bc81c64bc4afec1eba6859a83f
NodeType.DOMAIN
dict_keys([NodeView(node_name='Epic He', verify_key=cc9381403088cd4590b307868bb29e461863aabcb988905a5135a5a060af61af)])
node_name='Epic He' verify_key=cc9381403088cd4590b307868bb29e461863aabcb988905a5135a5a060af61af
Noerr before policy transformations
Noerr


```python
class Request:
  id: str = 997495402b644d29815421ab996ff7b3
  requesting_user_verify_key: str = cc9381403088cd4590b307868bb29e461863aabcb988905a5135a5a060af61af
  approving_user_verify_key: str = None
  request_time: str = 2023-03-16 18:05:11
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 79cec0cdd5b141808081ee28e83d265d
  request_hash: str = "e3e63628f4f803b56ac986e834b94b41427ca59e2533ec5e7f882f27726ce833"
  changes: str = [syft.core.node.new.request.UserCodeStatusChange]

```

In [ ]:
client.api.services.policy.get_all()[0]

```python
class UserPolicy:
  id: str = c6c4a8bc81c64bc4afec1eba6859a83f
  user_verify_key: str = cc9381403088cd4590b307868bb29e461863aabcb988905a5135a5a060af61af
  raw_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    # from syft.core.node.new.action_object import ActionObject
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  parsed_code: str = "@serializable(recursive_serde=True)
class RepeatedCallPolicy():
    # from syft.core.node.new.action_object import ActionObject
    from typing import Dict
    from typing import Any
    
    def __init__(self, n_calls=1, downloadable_output_args=[]):
        self.n_calls = n_calls + 1
        self.downloadable_output_args = downloadable_output_args
        self.state = {"counts": 0}
        # super.__init__(n_calls=n_calls, downloadable_output_args=downloadable_output_args)

    def __repr__(self) -> str:
        return f'RepeatedCallPolicy - {self.n_calls}/{self.downloadable_output_args}/{self.state}'

    def public_state(self):
        return self.state["counts"]
        
    def apply_output(self, action_object) -> Dict[Any, Any]:
        results_dict = action_object.syft_action_data        
        if self.state["counts"] < self.n_calls:
            for output_arg in self.downloadable_output_args:
                # results_dict[output_arg] = results_dict[output_arg].set_permission(READ, self.my_verify_key)
                results_dict[output_arg] = results_dict[output_arg]

            self.state["counts"] += 1
        else:
            return 
            
        return results_dict
"
  signature: str = (downloadable_output_args)
  class_name: str = "RepeatedCallPolicy"
  unique_name: str = "user_func_RepeatedCallPolicy_cc9381403088cd4590b307868bb29e461863aabcb988905a5135a5a060af61af_c2b88b38b612f1aa9f2c57ab61c711a8332435b895ebddf906d2ba84c183a6b7"
  code_hash: str = "c2b88b38b612f1aa9f2c57ab61c711a8332435b895ebddf906d2ba84c183a6b7"
  status: str = UserPolicyStatus.SUBMITTED

```

In [ ]:
func_plus_one

```python
class SubmitUserCode:
  id: str = None
  code: str = "@sy.syft_function(
    input_policy=ExactMatch(x=x_pointer),
    # input_policy_init_args={'inputs':{'x':4}},
    # output_policy=SingleExecutionExactOutput,
    # output_policy_init_args={},
    output_policy=output_policy.id,
    output_policy_init_args={"n_calls":1, 'downloadable_output_args':['y']},
    outputs=["y"]
)
def func_plus_one(x):
    return {"y": x + 1}
"
  func_name: str = "func_plus_one"
  signature: str = (x)
  input_policy: str = syft.core.node.new.user_code.ExactMatch
  input_policy_init_args: str = None
  output_policy: str = c6c4a8bc81c64bc4afec1eba6859a83f
  output_policy_init_args: str = {'n_calls': 1, 'downloadable_output_args': ['y']}
  local_function: str = <function func_plus_one at 0x7f0ef09dcd30>
  input_kwargs: str = ['x']
  outputs: str = ['y']
  enclave_metadata: str = None

```

In [ ]:
client.api.services.code.func

<function syft.core.node.new.api.generate_remote_function.<locals>.wrapper(*args, **kwargs)>

In [ ]:
client = sy.login(node=worker, email="info@openmined.org", password="changethis", cache=False)

Logged into Epic He as <info@openmined.org>


In [ ]:
client.notifications[-1].link.changes[0].link

ValidationError: 2 validation errors for UserCode
output_policy_state
  'utf-8' codec can't decode byte 0xca in position 100: invalid continuation byte (type=value_error.unicodedecode)
output_policy_state
  value is not a valid dict (type=type_error.dict)

In [ ]:
request = client.notifications[-1].link
func = request.changes[0].link
result = func.unsafe_function(x=x_pointer)
# func.output_policy
final_result = request.accept_by_depositing_result(result)
final_result

ValidationError: 2 validation errors for UserCode
output_policy_state
  'utf-8' codec can't decode byte 0xca in position 100: invalid continuation byte (type=value_error.unicodedecode)
output_policy_state
  value is not a valid dict (type=type_error.dict)

In [ ]:
client.notifications[-1].link.changes[-1]

AttributeError: 'ActionStoreChange' object has no attribute 'link'

In [ ]:
client.api.services.code.func_plus_one(x=x_pointer.id)

AttributeError: 'APIModule' object has no attribute 'func_plus_one'

In [ ]:
client.api.services.code.get_all()

: 

In [ ]:
func_plus_one = client.api.services.code.get_all()[-1]

: 

In [ ]:
print(func_plus_one.output_policy.raw_code)

: 

In [ ]:
state = sy.deserialize(func_plus_one.output_policy_state, from_bytes=True, class_type=RepeatedCallPolicy)

: 

In [ ]:
state.public_state()

: 